# Get Asset Data — Naperville Example

___

**Notebook Features:**

1. Extract all segments, corridors, or regions for a given territory

2. Get all segments for a given corridor

3. Extract all assets by any hierarchy level

4. Get detailed information about a given asset

___

**Example below is with Naperville** — Expected asset count:

`TERRITORY` count = 1

`REGION` count = 0

`CORRIDOR` count = 4

`SEGMENT` count = 764

___

Generate a token following the directions [here](https://ibm.github.io/Environmental-Intelligence-Suite/api-tokens.html) and paste the token into the quotes for `AUTH_TOKEN`.

In [1]:
AUTH_TOKEN = ''

___

## Import Libraries and Define Functions

In [2]:
import os
import json
import pprint
import requests
from IPython.display import Image
from datetime import datetime as dt

In [3]:
API_ENDPOINT = 'foundation.agtech.ibm.com/v2'
os.environ["API_ENDPOINT"] = API_ENDPOINT
os.environ["AUTH_TOKEN"]= AUTH_TOKEN

ASSET_MGMT = {
    'host': os.environ.get('API_ENDPOINT', 'Run the notebook to set env vars')
}

HEADERS = {
    'Accept': 'application/json',
    'Authorization': 'Bearer ' + os.environ.get("AUTH_TOKEN", 'Run the notebook to set env vars'),
    'Content-Type': 'application/json; charset=UTF-8'
}

PRINTER = pprint.PrettyPrinter(indent = 2)

In [4]:
def getHierarchy():
    get_url = 'https://{}/hierarchy'.format(ASSET_MGMT['host'])
    
    r = requests.get(url=get_url, headers=HEADERS)
    return json.loads(r.text) if (r.status_code == 200) else r.raise_for_status()



def getAsset(input_type=None, parentID=None, limit=1000, start=0, idsOnly=False, includeScalars=False, includeGeom=False, assets=None, firstRun=True, assetCount=None, showCount=True):
    
    if firstRun:
        assetCount = 0
        assets = {
            'features': []
        }
    
    params = {
        'start': start,
        'idsOnly': idsOnly,
        'includeAssetGeometry': includeGeom
    }
    
    if input_type is not None:
        params['inputType'] = input_type
        
    if parentID is not None:
        params['parentReferenceUuid'] = parentID

    if includeScalars is True:
        params['includeScalarInfo'] = includeScalars

    get_url = ('https://{}/asset?projection=4326&limit=' + str(limit)).format(ASSET_MGMT['host'])
    r = requests.get(url=get_url, headers=HEADERS, params=params)
    
    if (r.status_code == 200):
        response = json.loads(r.text)
        assets['features'].extend(response['features'])
        assetCount = assetCount + len(response['features'])

        if showCount:
            print('Assets retrieved so far:', assetCount)
        
        if len(response['features']) < limit:
            return assets
        
        else:
            return getAsset(input_type=input_type, parentID=parentID, limit=limit, start=start+limit, idsOnly=idsOnly, includeScalars=includeScalars, includeGeom=includeGeom, assets=assets, firstRun=False, assetCount=assetCount, showCount=showCount)
    
    else:
        r.raise_for_status()



def getAssetById(asset_id, projection = 4326, includeScalarInfo=False):
    get_url = 'https://{}/asset/{}?projection={}&includeScalarInfo={}'.format(ASSET_MGMT['host'], asset_id,projection,includeScalarInfo)
    
    r = requests.get(url=get_url, headers=HEADERS)
    return json.loads(r.text) if (r.status_code == 200) else r.raise_for_status()

___

___

## Get hierarchy data to determine how many hierarchy levels exist

In [40]:
hierarchy = getHierarchy()

In [41]:
hierarchyLevels = []
hierarchyCount = hierarchy['hierarchyList'][0]['numberOfLevels']
print('Number of Levels in Hierarchy:', hierarchyCount)
for level in hierarchy['hierarchyList'][0]['hierarchyLevels']:
    hierarchyLevels.append(level['key'])
    print(level['key'])

Number of Levels in Hierarchy: 3
TERRITORY
CORRIDOR
SEGMENT


___

## 1. Extract all segments, corridors, or regions for a given territory

**First, get a list of all territories**

In [42]:
territories = getAsset('TERRITORY')

Assets retrieved so far: 1


In [43]:
print('Total number of territories:', len(territories['features']))

Total number of territories: 1


In [44]:
count = 0
for territory in territories['features']:
    print('Territory Name:', territory['properties']['name'])
    print('Territory UUID:', territory['uuid'])
    print('Index:', count, '\n')
    count = count + 1

Territory Name: UTILITY
Territory UUID: 7dcd9bf6-e382-4a83-b10f-0f3be27860de
Index: 0 



**Next, select a territory.**

The default uses the first UUID in the list above. Uncomment the first line of code to specify the UUID or replace the `0` below with the index from the specified territory above.

In [45]:
# territory_uuid = '<Paste Territory UUID From Above>'
territory_uuid = territories['features'][0]['uuid']

In [46]:
territory_uuid

'7dcd9bf6-e382-4a83-b10f-0f3be27860de'

___

___

### 1a. Get all regions in specified territory (If applicable)

In [12]:
if 'REGION' in hierarchyLevels:
    regions = getAsset('REGION', territory_uuid)
    print('\nTotal number of regions:', len(regions['features']))
elif 'REGION' not in hierarchyLevels:
    print('Region hierarchy level does not exist')

Region hierarchy level does not exist


In [13]:
region_uuids = []
if 'REGION' in hierarchyLevels:
    for region in regions['features']:
        print('Region Name:', region['properties']['name'])
        print('Region UUID:', region['uuid'], '\n')
        region_uuids.append(region['uuid'])
elif 'REGION' not in hierarchyLevels:
    print('Region hierarchy level does not exist')

Region hierarchy level does not exist


___

### 1b. Get all corridors in specified territory
Iterate through all regions in the given territory to get all corridors

In [14]:
corridor_count = 0
corridor_uuids = []
corridors_in_territory = []

if hierarchyCount == 3:
    print('Retrieving corridors...\n')
    corridors_in_territory = getAsset('CORRIDOR', territory_uuid)
    for corridor in corridors_in_territory['features']:
        corridor_uuids.append(corridor['uuid'])

elif hierarchyCount == 4:
    print('Retrieving corridors...\n')
    for region_uuid in region_uuids:
        corridors = getAsset('CORRIDOR', region_uuid, showCount=False)
        corridor_count = corridor_count + len(corridors['features'])
        print('Corridors retrieved so far:', corridor_count)
        for corridor in corridors['features']:
            corridors_in_territory.append(corridor)
            corridor_uuids.append(corridor['uuid'])

print('\nTotal corridors retrieved:', len(corridor_uuids))

Retrieving corridors...

Assets retrieved so far: 4

Total corridors retrieved: 4


In [16]:
PRINTER.pprint(corridors_in_territory)

___

### 1c. Get all segments in specified territory

In [17]:
segments_in_territory = []
segment_uuids = []
segment_count = 0

print('Retrieving segments...\n')
for corridor_uuid in corridor_uuids:
    segments = getAsset('SEGMENT', corridor_uuid, showCount=False)
    segment_count = segment_count + len(segments['features'])
    print('Segments retrieved so far:', segment_count)
    for segment in segments['features']:
        segments_in_territory.append(segment)
        segment_uuids.append(segment['uuid'])
print('\nTask complete!')

Retrieving segments...

Segments retrieved so far: 178
Segments retrieved so far: 606
Segments retrieved so far: 616
Segments retrieved so far: 764

Task complete!


In [18]:
print('Total number of segments returned:', len(segments_in_territory))

Total number of segments returned: 764


In [20]:
PRINTER.pprint(segments_in_territory)

___

___

## 2. Get all segments for a given corridor

**Select a corridor**

The default uses the first UUID in the `corridor_uuids` list. Uncomment the first line of code to specify the UUID or replace the 0 below with the index from the specified corridor from the `corridor_uuids` list.

In [21]:
# corridor_uuids

In [22]:
# corridor_uuid = '<Paste Corridor UUID From Above>'
corridor_uuid = corridor_uuids[0]

In [23]:
corridor_segments_uuid = []

corridor_segments = getAsset('SEGMENT', corridor_uuid)

for segment in corridor_segments['features']:
    corridor_segments_uuid.append(segment['uuid'])

print('\nTotal number of segments retrieved:', len(corridor_segments_uuid))

Assets retrieved so far: 178

Total number of segments retrieved: 178


In [25]:
PRINTER.pprint(corridor_segments_uuid)

In [ ]:
PRINTER.pprint(corridor_segments)

___

___

## 3. Extract all assets by any hierarchy level

See all hierarchy levels available

In [26]:
PRINTER.pprint(hierarchyLevels)

['TERRITORY', 'CORRIDOR', 'SEGMENT']


___

## 3a. Get all territories
See output from API call made at the very beginning of the notebook

In [27]:
PRINTER.pprint(territories)

{ 'features': [ { 'projection': 4326,
                  'properties': { 'OWNER': 'UTILITY',
                                  'UNIQUE_ROW_ID_COLS': 'OWNER',
                                  'area': 15880107.934868898,
                                  'box': { 'east': -88.126621,
                                           'north': 41.801992,
                                           'south': 41.768295,
                                           'west': -88.206035},
                                  'centroid': { 'latitude': 41.78376495263274,
                                                'longitude': -88.16215815465807},
                                  'deleted': False,
                                  'geometryBufferSize': 0.0,
                                  'hierarchyLevelType': 'TERRITORY',
                                  'ianaTimeZone': 'America/Chicago',
                                  'inputType': 'ASSET',
                                  'name': 'UTILITY',
       

---

## 3b. Get all regions

In [28]:
if 'REGION' not in hierarchyLevels:
    print('This client does not have any regions')
else:
    all_regions = getAsset('REGION')
    print('\nTotal regions retrieved:', len(list(all_regions['features'])))
#     PRINTER.pprint(all_regions)

This client does not have any regions


___

## 3c. Get all corridors

In [30]:
start = dt.now()
all_corridors = getAsset('CORRIDOR')
print('\nTotal time required to retrieve all corridors:', dt.now()-start)

Assets retrieved so far: 4

Total time required to retrieve all corridors: 0:00:00.330777


In [31]:
print('Total regions retrieved:', len(list(all_corridors['features'])))

Total regions retrieved: 4


In [ ]:
PRINTER.pprint(all_corridors)

___

## 3d. Get all segments

In [33]:
start = dt.now()
all_segments = getAsset('SEGMENT')
print('\nTotal time required to retrieve all segments:', dt.now()-start)

Assets retrieved so far: 764

Total time required to retrieve all segments: 0:00:00.801053


In [34]:
print('Total segments retrieved:', len(list(all_segments['features'])))

Total segments retrieved: 764


In [36]:
PRINTER.pprint(all_segments)

___

___

## 4. Get Detailed Information About A Given Asset

Get Asset by UUID. Please specify the UUID of the asset of interest. The current default uses the first territory in the list above.

In [37]:
asset_uuid = territories['features'][0]['uuid']

In [38]:
detailed_asset_info = getAssetById(asset_uuid)

In [39]:
detailed_asset_info

{'uuid': '7dcd9bf6-e382-4a83-b10f-0f3be27860de',
 'properties': {'name': 'UTILITY',
  'area': 15880107.934868898,
  'box': {'north': 41.801992,
   'south': 41.768295,
   'east': -88.126621,
   'west': -88.206035},
  'centroid': {'latitude': 41.78376495263274, 'longitude': -88.16215815465807},
  'ianaTimeZone': 'America/Chicago',
  'deleted': False,
  'inputType': 'ASSET',
  'hierarchyLevelType': 'TERRITORY',
  'profileId': '960b1744-5527-4144-b4a1-05ea7497dbb2',
  'geometryBufferSize': 0.0,
  'OWNER': 'UTILITY',
  'UNIQUE_ROW_ID_COLS': 'OWNER'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-88.201452, 41.768295],
    [-88.196398, 41.768305],
    [-88.196383, 41.768305],
    [-88.193688, 41.768336],
    [-88.193573, 41.768342],
    [-88.186028, 41.768972],
    [-88.175021, 41.769892],
    [-88.161278, 41.771041],
    [-88.153957, 41.771653],
    [-88.144312, 41.772459],
    [-88.142899, 41.772577],
    [-88.141566, 41.772689],
    [-88.140093, 41.772812],
    [-88.138672, 41.772

___

___